## Module 1.3

In [2]:
import minsearch

In [21]:
import json

In [22]:
with open('documents.json',  'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields=['course']
)

In [26]:
q = 'the course has already started, can I still enroll?'

In [27]:
index.fit(documents)

In [28]:
# default boost of 1.0
boost = {
    'question': 3.0,
    'section': 0.5
}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course': 'data-engineering-zoomcamp'}
    
)

In [29]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Module 1.4

In [30]:
from google import genai
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

client = genai.Client(api_key=GEMINI_API_KEY)

In [31]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=q
)

In [32]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Whether or not you can still enroll in a course that has already started depends on a number of factors:\n\n*   **The specific course and institution:** Different schools, universities, online platforms, and organizations have different policies. Some allow late enrollment, others don't.\n*   **How far into the course it is:** A course that has just started might be more lenient than one that is halfway through.\n*   **Availability of spots:** If the course is already full, even if they allow late enrollment, you might not be able to get in.\n*   **Prerequisites:** If you need to catch up on a lot of missed material or fulfill specific requirements before joining, they might not allow it.\n*   **Instructor approval:** Somet

In [33]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [34]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

In [35]:
prompt = prompt_template.format(question=q, context=context).strip()

In [36]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=prompt
)

In [37]:
response.text

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n"

## Module 1.5

In [52]:
def search(query):
    # default boost of 1.0
    boost = {
        'question': 3.0,
        'section': 0.5
    }
    
    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=10,
        filter_dict={'course': 'data-engineering-zoomcamp'}
        
    )

    return results

In [54]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT does not contain the answer, output NONE
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """
    
    context = ""

    for doc in results:
        context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [47]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        # config=types.GenerateContentConfig(
            # system_instruction="You are a cat. Your name is Neko."),
        contents=prompt
    )
    return response.text

In [56]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer


In [59]:
rag('Can I follow the course on Windows?')

"Yes, you can follow the course on Windows. The path for activating the virtual environment is slightly different, it's env/Scripts/activate.\n"

## Module 1.6

In [70]:
from elasticsearch import Elasticsearch

es_url = 'https://legendary-space-guacamole-4j9ggwj59vqh5jgw-9200.app.github.dev/'

es_client = Elasticsearch(es_url)

In [71]:
es_client.info()

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Content-Type, Accept]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)